<a href="https://colab.research.google.com/github/Hamedsamak66/Bitcoin-Price-Prediction/blob/main/BTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from statsmodels.tsa.arima.model import ARIMA

# Step 1: Read and preprocess the dataset
df = pd.read_csv('/content/sample_data/Dataset.csv')
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:(i + seq_length), :])
        y.append(data[i + seq_length, -1])
    return np.array(X), np.array(y)

seq_length = 60  # Use 60 days of data to predict the next day's BTC price
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Step 3: Evaluate the LSTM model
y_pred_lstm = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred_lstm = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_lstm.shape[0], df.shape[1] - 1)), y_pred_lstm), axis=1))[:, -1]
y_test_lstm = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], df.shape[1] - 1)), y_test.reshape(-1, 1)), axis=1))[:, -1]

# Calculate R-squared and MSE for LSTM model
r2_lstm = r2_score(y_test_lstm, y_pred_lstm)
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)

print(f'LSTM Model - R-squared: {r2_lstm}, MSE: {mse_lstm}')

# Step 4: Build and train the Linear Regression model
# Reshape data for Linear Regression
X_lr = scaled_data[:, :-1]  # Use all columns except the last one (BTC price) for features
y_lr = scaled_data[:, -1]   # Use BTC price as the target

# Split the data into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_lr, test_size=0.2, random_state=42)

# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_lr, y_train_lr)

# Step 5: Evaluate the Linear Regression model
y_pred_lr = lr_model.predict(X_test_lr)

# Inverse transform the predictions and actual values
y_pred_lr = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_lr.shape[0], df.shape[1] - 1)), y_pred_lr.reshape(-1, 1)), axis=1))[:, -1]
y_test_lr = scaler.inverse_transform(np.concatenate((np.zeros((y_test_lr.shape[0], df.shape[1] - 1)), y_test_lr.reshape(-1, 1)), axis=1))[:, -1]

# Calculate R-squared and MSE for Linear Regression model
r2_lr = r2_score(y_test_lr, y_pred_lr)
mse_lr = mean_squared_error(y_test_lr, y_pred_lr)

print(f'Linear Regression Model - R-squared: {r2_lr}, MSE: {mse_lr}')

# Step 6: Build and train the ARIMA model
# ARIMA model works on univariate time series, use only the BTC price column
btc_prices = df['BTC'].values

# Split the data into training and testing sets
train_size = int(len(btc_prices) * 0.8)
train_btc, test_btc = btc_prices[:train_size], btc_prices[train_size:]

# Train the ARIMA model
arima_order = (5, 1, 0)  # Example order, you may need to tune this
arima_model = ARIMA(train_btc, order=arima_order)
arima_model_fit = arima_model.fit()

# Step 7: Evaluate the ARIMA model
pred_start = len(train_btc)
pred_end = len(btc_prices) - 1
y_pred_arima = arima_model_fit.predict(start=pred_start, end=pred_end, dynamic=False)

# Calculate R-squared and MSE for ARIMA model
r2_arima = r2_score(test_btc, y_pred_arima)
mse_arima = mean_squared_error(test_btc, y_pred_arima)

print(f'ARIMA Model - R-squared: {r2_arima}, MSE: {mse_arima}')


ValueError: could not convert string to float: 'Fear'

ValueError: could not convert string to float: 'Fear'